In [2]:
import re
import codecs
import sys
import pickle
#import time

In [2]:
from __future__ import print_function
def warning(*objs):
    print("WARNING: ", *objs, file=sys.stderr)

In [5]:
nomFeatures="NGrams-LetterFeatures.csv"
with codecs.open(nomFeatures,"r",encoding="utf8") as stream:
    featureLines=stream.readlines()
letterFeatures={}
for featureLine in featureLines[2:]:
    elements=featureLine.strip().split("\t")
    letterFeatures[elements[0]]=elements[1:]
#letterFeatures

In [6]:
def vectorizeNGram(ngram):
    vecteurs=[]
    for letter in ngram:
        vecteurs.append(letterFeatures[letter])
    return vecteurs

In [7]:
vectorizeNGram("bac")

[[u'0',
  u'0',
  u'1',
  u'1',
  u'0',
  u'1',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'1',
  u'0'],
 [u'1',
  u'1',
  u'0',
  u'0',
  u'0',
  u'1',
  u'0',
  u'1',
  u'0',
  u'0',
  u'0',
  u'0',
  u'1',
  u'1',
  u'0'],
 [u'0',
  u'0',
  u'1',
  u'0',
  u'0',
  u'2',
  u'1',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0',
  u'0']]

In [3]:
nomFichier="texte.txt"
try:
    fichier = codecs.open(nomFichier, "r",encoding="utf8")
except IOError:
    warning("could not open", nomFichier)
    sys.exit()
else:
    text = fichier.readlines()
    fichier.close()

In [4]:
ngram = 5
pc = [".", ",", ";", ":", "?", "!", "--", "..."]

In [5]:
def normaliserTexte(text,nospace=False):
    normalText=[]
    for line in text:
        for ponct in pc: 
            line=line.replace(ponct,"")
        line=line.replace("'"," ")
        line=line.replace("-"," ")
        if nospace:
            line=re.sub('\s+','',line)
        else:
            line=re.sub('\s+',' ',line)
        normalText.append(line.lower())
    return normalText

def extraireNgrams(text,noclasses=False):
    ngrams=[]
    classes=[]
    for line in text[:]:
        line = line.strip()
    #    print (line)
        espace = line.count(" ")
        i = 0
        while i <= len(line) - ngram - espace:
            if line[i] != " ":
                lettres = 0
                lettreslocales = 0
                ngramLocal=[]
                j = 0
                type = ""
                while lettres < ngram:
                    if line[i + j] != " ":
                        ngramLocal.append(line[i + j])
                        lettres += 1
                        lettreslocales += 1
                    else:
                        type = type + str(lettreslocales) + "_"
                        lettreslocales = 0
                    j += 1
                type = type + str(lettreslocales)
                ngrams.append(ngramLocal)
                classes.append(type)
            i = i + 1
    if noclasses:
        return ngrams
    else:
        return(ngrams, classes)

In [6]:
trainTexte = normaliserTexte(text)
(ngramTrain,classesTrain)=extraireNgrams(trainTexte)
print (ngramTrain[0],classesTrain[0])

[u'c', u'h', u'a', u'p', u'i'] 5


In [7]:
#testSource=[u"Dans cette histoire, il y a un loup, une chêvre, et un agneau. Tout se passe bien tant que le loup et l'agneau ne sont pas seuls."]

In [8]:
nomFichier="LesMiserables.txt"
try:
    fichier = codecs.open(nomFichier, "r", "utf-8")
except IOError:
    warning("could not open", nomFichier)
    sys.exit()
else:
    testSource = fichier.readlines()
    fichier.close()

In [9]:
testTexte=normaliserTexte(testSource,True)
ngramTest=extraireNgrams(testTexte,True)
print (ngramTest[0])
ngramComplete=ngramTrain+ngramTest
textComplete=["".join(x) for x in ngramComplete]
textTrain=["".join(x) for x in ngramTrain]
textTest=["".join(x) for x in ngramTest]
print (len(textComplete),len(textTrain),len(textTest))
print (textComplete[0],textTrain[0],textTest[0])

[u'\ufeff', u't', u'o', u'm', u'e']
2327848 100405 2227443
chapi chapi ﻿tome


CountVectorizer permet de traiter le texte comme des données numériques  
- analyzer permet de définir le niveau de tokenization (word, char)
- ngram_range permet de définir les ngrams qui seront extraits (min, max)

La méthode fit_transform permet de déterminer les données numériques (features) qui seront utilisées pour représenter le texte.

Ensuite, on peut utiliser la méthode transform pour représenter un texte en utilisant ces features.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
transformVectors = CountVectorizer(analyzer=u'char',ngram_range=(5, 5))
skComplete = transformVectors.fit_transform(textComplete)

SVM est une des formes d'apprentissage
- gamma définit le nombre de features qui seront utilisées pour l'apprentissage
- C ???

In [27]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

skTrain est la représentation numérique de textTrain

clf.fit permet de faire l'apprentissage du lien entre les données de skTrain et les étiquettes fournies dataClasses

In [28]:
%%time
skTrain=transformVectors.transform(textTrain)
clf.fit(skTrain, classesTrain)

CPU times: user 14min 6s, sys: 18.1 s, total: 14min 24s
Wall time: 14min 24s


In [29]:
with open('SVC-Decoupages.pkl', 'wb') as output:
   pickle.dump(clf, output, pickle.HIGHEST_PROTOCOL)

skTest est la représentation numérique de textTest

clf.predict permet de faire la prédiction des étiquettes pour les données de skTest

In [30]:
skTest=transformVectors.transform(textTest)
prediction=clf.predict(skTest)

##interprétation des résultats

###Découpage des ngrams

découpage des ngrams de test en fonction des étiquettes => resultChunks

In [31]:
resultChunks=[]
for i,element in enumerate(prediction):
    chunks=[int(n) for n in element.split("_")]
#    print (chunks)
    debut=0
    elementChunks=[]
    for chunk in chunks:
        elementChunks.append(textTest[i][debut:debut+chunk])
#        print (textTest[i][debut:debut+chunk],end=" ")
        debut+=chunk
#    print ()
    resultChunks.append(elementChunks)
#print (resultChunks)

###Calcul des votes

calcul des votes pour les séparations entre mots

In [32]:
testSpaces={}
for i,chunks in enumerate(resultChunks):
    posChunk=0
    for chunk in chunks:
        if posChunk!=0: 
#            print (i+posChunk,chunk)
            if not i+posChunk in testSpaces:
                testSpaces[i+posChunk]=0
            testSpaces[i+posChunk]+=1
        posChunk+=len(chunk)
#testSpaces

###Découpage de test

Séparation des mots

In [33]:
for i,lettre in enumerate(testTexte[200:250]):
    if i in testSpaces:
        # choisir le seuil d'apparition des espaces
        # au dessus de deux types convergents : >2
        # on insère un espace
        if testSpaces[i]>2:
            print ("_",end="")
        # sinon, par exemple, on colle les lettres
        # on pourrait aussi mettre le nombre de votes convergents
        # pour voir ce que ça donne
        else:
            pass
#            print (testSpaces[i],end="")
    print (lettre,end="")
#    time.sleep(1)

monseigneurmademoisellebaptistineétaitunepersonnelonguepâlemincedouceelleréalisaitlidéaldecequexprimelemot«respectable»carilsemblequilsoitnécessairequunefemmesoitmèrepourêtrevénérableellenavaitjamaisétéjolietoutesaviequinavaitétéquunesuitedesaintesoeuvresavaitfiniparmettresurelleunesortedeblancheuretdeclartéetenvieillissantelleavaitgagnécequonpourraitappelerlabeautédelabontécequiavaitétédelamaigreurdanssajeunesseétaitdevenudanssamaturitédelatransparenceetcettediaphanéitélaissaitvoirlangecétaituneâmeplusencorequecenétaituneviergesapersonnesemblaitfaitedombreàpeineassezdecorpspourquilyeûtlàunsexeunpeudematièrecontenantunelueurdegrandsyeuxtoujoursbaissésunprétextepourquuneâmerestesurlaterre_madamemagloireétaitunepetitevieilleblanchegrassereplèteaffairéetoujourshaletanteàcausedesonactivitédabordensuiteàcausedunasthmeàsonarrivéeoninstallammyrielensonpalaisépiscopalavecleshonneursvoulusparlesdécretsimpériauxquiclassentlévêqueimmédiatementaprèslemaréchaldecamplemaireetleprésidentluifirentlapr